In [2]:
import pandas as pd
import re
import sys
import importlib.util
import itertools
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder



%matplotlib inline

In [209]:
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('data/data (1).csv')

In [219]:
df = df[df['price'] < 180000]


In [4]:
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [253]:
df.dtypes.to_frame().T

,char_ar,char_en,char1_ar,char2_ar,char3_ar,char1_en,char2_en,char3_en,plate_no,price,class,plate_no_length,similar_digits,sepcial_date,saudi_tribes,One Digit,Two Digits (Same),Two Digits (Different),Three Digits (Same),Three Digits (Different),Four Digits (Same),Four Digits (Different),emergency_no,in_order,reversed_order,same_middles_no,plaindromic_no,same_first_last_no,same_two_sides,one_digit_zero,one_digit_one,one_digit_two,one_digit_three,one_digit_four,one_digit_five,one_digit_six,one_digit_seven,one_digit_eight,one_digit_nine,is_triple_letters,is_one_letters,First_Third_Match,is_sequence,Contains_Tribe,contains_special_words,contains_special_cars,connect_12,connect_23,combined_word,word_freq_score,first_name_rank,result
0,object,object,object,object,object,object,object,object,int64,int64,object,int64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,object,float64,float64,int32


# Build Model

In [5]:
# one hot coding
df_Model = pd.get_dummies(df)

In [6]:
df_Model.shape

(2528, 5032)

In [7]:
correlation = df_Model.select_dtypes(include=['number']).corr()
print(correlation['price'].sort_values(ascending=False))

price                       1.000000
One Digit                   0.531183
in_order                    0.499444
reversed_order              0.495302
similar_digits              0.432810
one_digit_one               0.422279
one_digit_two               0.239852
one_digit_three             0.224914
one_digit_nine              0.117018
one_digit_five              0.116424
one_digit_seven             0.101721
one_digit_eight             0.089098
one_digit_four              0.084941
one_digit_six               0.068741
Two Digits (Same)           0.063770
Three Digits (Same)         0.026733
Two Digits (Different)      0.011358
First_Third_Match           0.007298
emergency_no                0.007025
is_triple_letters           0.003022
word_freq_score            -0.001026
contains_special_words     -0.009860
sepcial_date               -0.015292
saudi_tribes               -0.019520
Contains_Tribe             -0.020354
plaindromic_no             -0.022210
Four Digits (Same)         -0.030093
f

In [8]:
# Set the correlation threshold
threshold = 0.1  # You can change this value based on your requirement

# Filter the correlations
# We use `abs()` for absolute value to consider both strong positive and negative correlations
selected_features = correlation[abs(correlation['price']) > threshold]['price'].index
selected_features

Index(['plate_no', 'price', 'plate_no_length', 'similar_digits', 'One Digit',
       'Four Digits (Different)', 'in_order', 'reversed_order',
       'same_middles_no', 'same_two_sides', 'one_digit_one', 'one_digit_two',
       'one_digit_three', 'one_digit_five', 'one_digit_seven',
       'one_digit_nine'],
      dtype='object')

In [9]:
features = [
        'price', 'plate_no_length', 'similar_digits', 'One Digit',
       'Two Digits (Same)', 'Two Digits (Different)', 'Three Digits (Same)',
       'Four Digits (Different)', 'in_order', 'reversed_order',
       'same_middles_no', 'one_digit_one', 'one_digit_two', 'one_digit_three',
       'one_digit_four', 'one_digit_five', 'one_digit_six', 'one_digit_seven',
       'one_digit_eight', 'one_digit_nine', 'connect_12', 'connect_23',
       'result'
]

df_Model = df_Model[features]




# Train

In [10]:
# Prepare data
X = df_Model.drop(['price'], axis=1)
y = df_Model['price']

In [11]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=44)

# sacle the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
X.shape

(2528, 22)

# Build Model

In [13]:
lasso_model = Lasso()  

In [14]:
param_grid = {
    'alpha': [0, 0.003, 0.0002, 0.001, 0.01, 0.5, 1, 5]  # Regularization strength
}

grid_search_lasso = GridSearchCV(estimator=lasso_model,
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error', 
                           verbose=1)

In [15]:
# Fit the model on the training data
lasso_model.fit(X_train_scaled, y_train)

Lasso()

In [16]:
# Fit GridSearchCV on the training data

grid_search_lasso.fit(X_train_scaled, y_train)

# Get the best parameters and best model


# Lasso
best_params_lasso = grid_search_lasso.best_params_
best_model_lasso = grid_search_lasso.best_estimator_

print("Best Parameters for lasso model:", best_params_lasso)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters for lasso model: {'alpha': 5}


In [17]:
# Predict and evaluate the model
y_pred_lasso = best_model_lasso.predict(X_test_scaled) # Lasso model

In [18]:
coeff_df_lasso = pd.DataFrame(best_model_lasso.coef_,X.columns,columns=['Coefficient']) # Lasso model cofficients

In [19]:
coeff_df_lasso

,Coefficient
plate_no_length,-7.882454e+03
similar_digits,1.310428e+03
One Digit,1.502301e+04
Two Digits (Same),3.631342e+03
Two Digits (Different),1.737636e+03
Three Digits (Same),3.523893e+03
Four Digits (Different),-1.320690e+03
in_order,-1.651673e+03
reversed_order,2.260320e+03
same_middles_no,-4.854285e+02


In [20]:
print(f" Lasso model intercept: {best_model_lasso.intercept_}") 

 Lasso model intercept: 32046.311572700295


In [21]:
y_base = [y_train.mean()] * len(y_test)

In [22]:
# our benchmark model
mse_base = mean_squared_error(y_test, y_base )
print(f'Mean Squared Error of base model: {mse_base:2e}')

# MSE Lasso model
mse_train_lasso = mean_squared_error(y_train, best_model_lasso.predict(X_train_scaled))
mse_test_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f'Mean Squared Error for training lasso model: {mse_train_lasso:2e}')
print(f'Mean Squared Error for testing lasso model: {mse_test_lasso:2e}')

# our benchmark model
mae_base = mean_absolute_error(y_test, y_base )
print(f'Mean Absolute  Error of base model: {mae_base:2e}')
# Lasso model
mae_train_lasso = mean_absolute_error(y_train, best_model_lasso.predict(X_train_scaled))
mae_test_lasso = mean_absolute_error(y_test, y_pred_lasso)
print(f'Mean Absolute Error for training lasso model: {mae_train_lasso:2e}')
print(f'Mean Absolute Error for testing lasso model: {mae_test_lasso:2e}')

# our benchmark model
print(f'Root Mean Squared  Error of base model: {np.sqrt(mse_base):2e}')
# Lasso model
mae_train_lasso = mean_absolute_error(y_train, best_model_lasso.predict(X_train_scaled))
mae_test_lasso = mean_absolute_error(y_test, y_pred_lasso)
print(f'Mean Absolute Error for training lasso model: {mae_train_lasso:2e}')
print(f'Mean Absolute Error for testing lasso model: {mae_test_lasso:2e}')

# our benchmark model
print(f'Root Mean Squared  Error of base model: {np.sqrt(mse_base):2e}')

# RMSE Lasso model

print(f'Root Mean Squared Error for training lasso model: {np.sqrt(mse_train_lasso):2e}')
print(f'Root Mean Squared Error for testing lasso model: {np.sqrt(mse_test_lasso):2e}')

# Lasso model r2 score
r2_score_train_lasso = r2_score(y_train, best_model_lasso.predict(X_train_scaled))
r2_score_test_lasso = r2_score(y_test, y_pred_lasso)
print(f'R Square for training lasso model: {r2_score_train_lasso}')
print(f'R Square for testing lasso model: {r2_score_test_lasso}')

mean_absolute_percentage_error_lasso = mean_absolute_percentage_error(y_train, best_model_lasso.predict(X_train_scaled))
mean_absolute_percentage_error_test_lasso = mean_absolute_percentage_error(y_test, y_pred_lasso)
print(f'percentage_error for training lasso model: {mean_absolute_percentage_error_lasso}')
print(f'percentage_error for testing lasso model: {mean_absolute_percentage_error_test_lasso}')

Mean Squared Error of base model: 1.196833e+09
Mean Squared Error for training lasso model: 1.756867e+09
Mean Squared Error for testing lasso model: 6.522052e+08
Mean Absolute  Error of base model: 2.201493e+04
Mean Absolute Error for training lasso model: 1.900356e+04
Mean Absolute Error for testing lasso model: 1.438921e+04
Root Mean Squared  Error of base model: 3.459527e+04
Mean Absolute Error for training lasso model: 1.900356e+04
Mean Absolute Error for testing lasso model: 1.438921e+04
Root Mean Squared  Error of base model: 3.459527e+04
Root Mean Squared Error for training lasso model: 4.191500e+04
Root Mean Squared Error for testing lasso model: 2.553831e+04
R Square for training lasso model: 0.403084056167501
R Square for testing lasso model: 0.434673712389529
percentage_error for training lasso model: 0.9759613682162779
percentage_error for testing lasso model: 0.8995345627354598


In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score







In [24]:
gb_model = GradientBoostingRegressor(n_estimators=163, random_state=42, max_depth= 3, min_samples_split= 5, min_samples_leaf=3, learning_rate = 0.07066694890477898)


cv_scores = cross_val_score(gb_model, X, y, cv=5, scoring='r2') 


print("Cross-validation R² scores:", cv_scores)
print("Mean R² score from cross-validation:", cv_scores.mean())
print("Standard deviation of R² scores:", cv_scores.std())




Cross-validation R² scores: [0.15204642 0.37422953 0.48751637 0.24517262 0.37845341]
Mean R² score from cross-validation: 0.3274836709013799
Standard deviation of R² scores: 0.11657348990886608


In [25]:
gb_model.fit(X_train, y_train)


y_pred_train_gb = gb_model.predict(X_train)
y_pred_test_gb = gb_model.predict(X_test)


print("Gradient Boosting - mean_squared_error score:", mean_squared_error(y_train, y_pred_train_gb))
print("Gradient Boosting - mean_squared_error score:", mean_squared_error(y_test, y_pred_test_gb))

print("Gradient Boosting - mean_absolute_error score:", mean_absolute_error(y_train, y_pred_train_gb))
print("Gradient Boosting - mean_absolute_error score:", mean_absolute_error(y_test, y_pred_test_gb))

print("Gradient Boosting - r2_score score:", r2_score(y_train, y_pred_train_gb))
print("Gradient Boosting - r2_score score:", r2_score(y_test, y_pred_test_gb))

Gradient Boosting - mean_squared_error score: 1738620036.7183921
Gradient Boosting - mean_squared_error score: 666931198.015626
Gradient Boosting - mean_absolute_error score: 18865.097098008668
Gradient Boosting - mean_absolute_error score: 14514.318540280656
Gradient Boosting - r2_score score: 0.409283822683456
Gradient Boosting - r2_score score: 0.4219093184403797


In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def objective(trial):
 
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    # Create the model with suggested parameters
    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    # Train the model
    model.fit(X_train, y_train)

    # Predict on test set
    y_pred = model.predict(X_test)

    # Calculate the mean squared error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    return mse  # We want to minimize the MSE

# Create the Optuna study
study = optuna.create_study(direction='minimize')

# Optimize the objective function (searching for the best hyperparameters)
study.optimize(objective, n_trials=100)

# Get the best trial (best hyperparameters)
best_trial = study.best_trial
print(f"Best Hyperparameters: {best_trial.params}")

# Train the model with the best hyperparameters
best_model = GradientBoostingRegressor(
    n_estimators=best_trial.params['n_estimators'],
    max_depth=best_trial.params['max_depth'],
    learning_rate=best_trial.params['learning_rate'],
    min_samples_split=best_trial.params['min_samples_split'],
    min_samples_leaf=best_trial.params['min_samples_leaf'],
    random_state=42
)

# Train the model on training data
best_model.fit(X_train, y_train)

# Predict on train and test sets
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

# Calculate MSE for both train and test sets
mse_train = mean_absolute_error(y_train, y_pred_train)
mse_test = mean_absolute_error(y_test, y_pred_test)

print(f"Mean Squared Error (MSE) - Train: {mse_train}")
print(f"Mean Squared Error (MSE) - Test: {mse_test}")



NameError: name 'X' is not defined

In [ ]:
import xgboost as xgb


xg_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xg_model.fit(X_train, y_train)


y_pred_train_xg = xg_model.predict(X_train)
y_pred_test_xg = xg_model.predict(X_test)


print("XGBoost - Training Mean Absolute Error:", mean_absolute_error(y_train, y_pred_train_xg))
print("XGBoost - Testing Mean Absolute Error:", mean_absolute_error(y_test, y_pred_test_xg))

print("XGBoost  - r2_score score:", r2_score(y_train, y_pred_train_gb))
print("XGBoost  - r2_score score:", r2_score(y_test, y_pred_test_gb))


XGBoost - Training Mean Absolute Error: 18805.03125
XGBoost - Testing Mean Absolute Error: 14470.2451171875
Gradient Boosting - r2_score score: 0.409283822683456
Gradient Boosting - r2_score score: 0.4219093184403797


============================================================
### Training model for plate_no_length = 1
  - Subset size: 268
  - Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
  - Train MSE: 5857987083.7429 | Test MSE: 8128764499.7954
  - Train MAE: 48153.4079 | Test MAE: 58194.6875
  - Train RMSE: 76537.4881 | Test RMSE: 90159.6612
  - Train R^2: 0.4940   | Test R^2: -0.2741

============================================================
### Training model for plate_no_length = 2
  - Subset size: 590
  - Best Params: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}
  - Train MSE: 1280837195.3274 | Test MSE: 995506763.7696
  - Train MAE: 18187.6288 | Test MAE: 18381.0698
  - Train RMSE: 35788.7859 | Test RMSE: 31551.6523
  - Train R^2: 0.1387   | Test R^2: -0.0490

============================================================
### Training model for plate_no_length = 3
  - Subset size: 899
  - Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
  - Train MSE: 632415122.6036 | Test MSE: 563257906.3105
  - Train MAE: 12367.8619 | Test MAE: 14835.2819
  - Train RMSE: 25147.8652 | Test RMSE: 23733.0551
  - Train R^2: 0.0556   | Test R^2: 0.0448

============================================================
### Training model for plate_no_length = 4
  - Subset size: 771
  - Best Params: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
  - Train MSE: 255184418.9718 | Test MSE: 73873503.3239
  - Train MAE: 6360.7831 | Test MAE: 5802.2115
  - Train RMSE: 15974.4928 | Test RMSE: 8594.9697
  - Train R^2: 0.3761   | Test R^2: -0.0159
  
============================================================